# 1. Instanciate URIs for terms (legislatura) 

# 3. Allocate Posts and Memberships

 Solves URI to Post as interval scheduling problem.

In [13]:
import json 
import pandas as pd
import sys
from collections import deque

start_date = '2015-02-01'

# converts xml 2 dataframe
from models.xml2dataframe import XML2DataFrame

# reads XML files 
import xml.etree.ElementTree as ET 
tree = ET.parse('datasets/ObterDeputados.xml') # api v1 this list is volatile and has 513 deputados
xmlstr_current_congressmen = ET.tostring(tree.getroot(), encoding='utf-8', method='xml')


In [14]:
with open('datasets/posts-55.csv', mode='r') as f:
    postsstr = f.read()
f.close()
posts = postsstr.split('\n')[1:]
postsdeque_dict = {start_date:deque(posts, maxlen=len(posts))} 
print(len(posts))

513


In [15]:
with open('datasets/membership_with_role_deputado-55.json', mode='r') as f:
    membership_with_role_deputado = json.load(f)
f.close()
df = pd.DataFrame.from_dict(membership_with_role_deputado)
df.head()


,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,None,2015-02-01,None,None,None,AP,Titular,0c09b717-9e84-4f96-baff-1396ffde2225,b27beba7-ca02-4041-a9e0-1793bcd141fe,f7dd38ab-56a0-4d4d-b336-1a6715c7a1d7
1,None,2015-02-01,None,None,None,RN,Titular,991898e8-c29b-48f3-b063-993647de89f4,b27beba7-ca02-4041-a9e0-1793bcd141fe,46745ddb-a65c-4b2f-864c-f605e6e359fa
2,None,2015-02-01,None,None,None,AC,Titular,8b6b072f-2ddb-42cb-bde3-d1aae3fb5610,b27beba7-ca02-4041-a9e0-1793bcd141fe,59cd5d76-8acd-420c-af62-85fcdba59337
3,None,2015-02-01,None,None,None,MA,Titular,8e0752c7-21d0-46b9-8fef-0a3a49305652,b27beba7-ca02-4041-a9e0-1793bcd141fe,8a1caba9-c718-4aab-a664-425ca788d487
4,None,2018-02-21,None,132056,None,SP,Suplente,e3424f93-3e6e-48a8-a6d6-8dc50138f1fc,b27beba7-ca02-4041-a9e0-1793bcd141fe,4b6d13d0-cd4e-41e6-922b-a1b188ed460b


In [16]:
def sortfinish(x):
    if not x['cam:dataFim']:
        return '1800-01-01'
    return x['cam:dataFim']

def sortstart(x):
    return x['cam:dataInicio']

def pop(process_date):
    key = min([k for k in postsdeque_dict 
               if ((k <= process_date) and (len(postsdeque_dict[k]) > 0))])
    this_post =  postsdeque_dict[key].pop()
    return this_post

def push(process_date, this_post):
    if process_date not in postsdeque_dict:
        postsdeque_dict[process_date] = deque([this_post] , maxlen=315)
    else:
        postsdeque_dict[process_date].append(this_post) 

def peeklen(process_date):
    keys = [k for k in postsdeque_dict 
               if ((k <= process_date) and (len(postsdeque_dict[k]) > 0))]
    return sum([len(postsdeque_dict[k]) for k in keys])

In [17]:
membership_with_role_deputado2 = sorted(
    sorted( membership_with_role_deputado,
            key=sortfinish),
    key=sortstart
)
df = pd.DataFrame.from_dict(membership_with_role_deputado2)

df.to_csv('datasets/membership_with_role_deputado-55.csv', sep=';', encoding='utf-8')
df.head()

,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,None,2015-02-01,None,None,None,AP,Titular,0c09b717-9e84-4f96-baff-1396ffde2225,b27beba7-ca02-4041-a9e0-1793bcd141fe,f7dd38ab-56a0-4d4d-b336-1a6715c7a1d7
1,None,2015-02-01,None,None,None,RN,Titular,991898e8-c29b-48f3-b063-993647de89f4,b27beba7-ca02-4041-a9e0-1793bcd141fe,46745ddb-a65c-4b2f-864c-f605e6e359fa
2,None,2015-02-01,None,None,None,AC,Titular,8b6b072f-2ddb-42cb-bde3-d1aae3fb5610,b27beba7-ca02-4041-a9e0-1793bcd141fe,59cd5d76-8acd-420c-af62-85fcdba59337
3,None,2015-02-01,None,None,None,MA,Titular,8e0752c7-21d0-46b9-8fef-0a3a49305652,b27beba7-ca02-4041-a9e0-1793bcd141fe,8a1caba9-c718-4aab-a664-425ca788d487
4,None,2015-02-01,None,None,None,SP,Titular,b7e9e1fb-c94d-4e3e-ace5-eae5336202d8,b27beba7-ca02-4041-a9e0-1793bcd141fe,7cb50203-c46c-48e3-9d0a-76fdb27baa48


In [18]:
total_posts = peeklen('2999-01-01')
_message = '{:}:processing_date:{:}\tnum_vacancies:{:}\tincoming:{:}\n'
with open('role_deputado_with_posts-55.csv', mode='w+') as f:
    f.write('org:role;org:post\n')
    
    for i, d in enumerate(membership_with_role_deputado2):
        processing_date = d['cam:dataInicio']
        finish_date = d['cam:dataFim']
        
        # pops from processing_date
        this_post = pop(processing_date)
        num_posts = peeklen(processing_date)
        incoming= peeklen('2999-01-01') - num_posts
        
        sys.stdout.write(_message.format(i + 1, processing_date, num_posts, incoming))
        # pushes on finish_date
        if finish_date:     
            push(finish_date, this_post)
                
        buff = '{:};{:}\n'.format(d['org:role'] , this_post)
        f.write(buff)


1:processing_date:2015-02-01	num_vacancies:512	incoming:0
2:processing_date:2015-02-01	num_vacancies:511	incoming:0
3:processing_date:2015-02-01	num_vacancies:510	incoming:0
4:processing_date:2015-02-01	num_vacancies:509	incoming:0
5:processing_date:2015-02-01	num_vacancies:508	incoming:0
6:processing_date:2015-02-01	num_vacancies:507	incoming:0
7:processing_date:2015-02-01	num_vacancies:506	incoming:0
8:processing_date:2015-02-01	num_vacancies:505	incoming:0
9:processing_date:2015-02-01	num_vacancies:504	incoming:0
10:processing_date:2015-02-01	num_vacancies:503	incoming:0
11:processing_date:2015-02-01	num_vacancies:502	incoming:0
12:processing_date:2015-02-01	num_vacancies:501	incoming:0
13:processing_date:2015-02-01	num_vacancies:500	incoming:0
14:processing_date:2015-02-01	num_vacancies:499	incoming:0
15:processing_date:2015-02-01	num_vacancies:498	incoming:0
16:processing_date:2015-02-01	num_vacancies:497	incoming:0
17:processing_date:2015-02-01	num_vacancies:496	incoming:0
18:pro

 # Deputados em exercício

In [19]:
xml2df = XML2DataFrame(xmlstr_current_congressmen)
df_current_congressman = xml2df.process_data()
df_current_congressman.head()

,anexo,codOrcamento,comissoes,condicao,deputado,email,fone,gabinete,idParlamentar,ideCadastro,matricula,nome,nomeParlamentar,partido,sexo,uf,urlFoto
0,4,2804,\n,Suplente,\n,dep.junjiabe@camara.gov.br,3215-5512,512,1635214,160544,610,JUNJI ABE,JUNJI ABE,PSD,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...
1,4,3611,\n,Titular,\n,dep.luizaerundina@camara.gov.br,3215-5620,620,5830288,74784,371,LUIZA ERUNDINA DE SOUSA,LUIZA ERUNDINA,PSOL,feminino,SP,http://www.camara.gov.br/internet/deputado/ban...
2,4,2615,\n,Titular,\n,dep.robertoalves@camara.gov.br,3215-5946,946,5830289,152610,387,BENEDITO ROBERTO ALVES FERREIRA,ROBERTO ALVES,PRB,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...
3,3,2429,\n,Titular,\n,dep.lazarobotelho@camara.gov.br,3215-5478,478,5830290,141480,63,LÁZARO BOTELHO MARTINS,LÁZARO BOTELHO,PP,masculino,TO,http://www.camara.gov.br/internet/deputado/ban...
4,4,2471,\n,Titular,\n,dep.sergiobrito@camara.gov.br,3215-5638,638,5830291,73808,213,SÉRGIO LUÍS LACERDA BRITO,SÉRGIO BRITO,PSD,masculino,BA,http://www.camara.gov.br/internet/deputado/ban...


 # Deputados SLP

In [20]:
df_agents = pd.read_csv('datasets/deputados-55.csv',sep=';',encoding='utf-8')
df_agents.head()

,cam:dataFalecimento,cam:dataNascimento,cam:email,cam:ideCadastro,cam:nomeCivil,cam:nomeParlamentarAtual,cam:nomeProfissao,cam:sexo,slp:old_resource_uri,slp:resource_uri
0,NaN,1989-02-26,dep.alielmachado@camara.gov.br,178927,ALIEL MACHADO BARK,ALIEL MACHADO,NaN,M,292752eeb282a9776ee443cf44b26bb7,15b8fc34-6360-46bc-8b4d-8ec80564101d
1,NaN,1973-11-11,dep.joaomarcelosouza@camara.gov.br,112437,JOÃO MARCELO SANTOS SOUZA,JOÃO MARCELO SOUZA,Psicólogo,M,de549e980bd660b3b287d5893a2e41a8,8e0752c7-21d0-46b9-8fef-0a3a49305652
2,NaN,1940-12-15,dep.junjiabe@camara.gov.br,160544,JUNJI ABE,JUNJI ABE,NaN,M,NaN,e3424f93-3e6e-48a8-a6d6-8dc50138f1fc
3,NaN,1966-06-22,dep.robertogoes@camara.gov.br,178995,ANTONIO ROBERTO RODRIGUES GÓES DA SILVA,ROBERTO GÓES,Administrador,M,d756776bc56efce11d2fecee1f4855d0,0c09b717-9e84-4f96-baff-1396ffde2225
4,NaN,1971-10-28,dep.alessandromolon@camara.gov.br,160511,ALESSANDRO LUCCIOLA MOLON,ALESSANDRO MOLON,NaN,M,069ec5b7ba726af033923cb2dd8a7340,e0bea418-a279-4fc1-be87-0ad12b13d570


In [21]:
df.head()

,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,None,2015-02-01,None,None,None,AP,Titular,0c09b717-9e84-4f96-baff-1396ffde2225,b27beba7-ca02-4041-a9e0-1793bcd141fe,f7dd38ab-56a0-4d4d-b336-1a6715c7a1d7
1,None,2015-02-01,None,None,None,RN,Titular,991898e8-c29b-48f3-b063-993647de89f4,b27beba7-ca02-4041-a9e0-1793bcd141fe,46745ddb-a65c-4b2f-864c-f605e6e359fa
2,None,2015-02-01,None,None,None,AC,Titular,8b6b072f-2ddb-42cb-bde3-d1aae3fb5610,b27beba7-ca02-4041-a9e0-1793bcd141fe,59cd5d76-8acd-420c-af62-85fcdba59337
3,None,2015-02-01,None,None,None,MA,Titular,8e0752c7-21d0-46b9-8fef-0a3a49305652,b27beba7-ca02-4041-a9e0-1793bcd141fe,8a1caba9-c718-4aab-a664-425ca788d487
4,None,2015-02-01,None,None,None,SP,Titular,b7e9e1fb-c94d-4e3e-ace5-eae5336202d8,b27beba7-ca02-4041-a9e0-1793bcd141fe,7cb50203-c46c-48e3-9d0a-76fdb27baa48


In [22]:
df2 = df.merge(df_agents, left_on='org:hasMember', right_on='slp:resource_uri', how='inner')
print(df2.shape[0])
df2.head()

729


,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri_x,cam:dataFalecimento,cam:dataNascimento,cam:email,cam:ideCadastro,cam:nomeCivil,cam:nomeParlamentarAtual,cam:nomeProfissao,cam:sexo,slp:old_resource_uri,slp:resource_uri_y
0,None,2015-02-01,None,None,None,AP,Titular,0c09b717-9e84-4f96-baff-1396ffde2225,b27beba7-ca02-4041-a9e0-1793bcd141fe,f7dd38ab-56a0-4d4d-b336-1a6715c7a1d7,NaN,1966-06-22,dep.robertogoes@camara.gov.br,178995,ANTONIO ROBERTO RODRIGUES GÓES DA SILVA,ROBERTO GÓES,Administrador,M,d756776bc56efce11d2fecee1f4855d0,0c09b717-9e84-4f96-baff-1396ffde2225
1,None,2015-02-01,None,None,None,RN,Titular,991898e8-c29b-48f3-b063-993647de89f4,b27beba7-ca02-4041-a9e0-1793bcd141fe,46745ddb-a65c-4b2f-864c-f605e6e359fa,NaN,1973-12-07,dep.felipemaia@camara.gov.br,141429,FELIPE CATALÃO MAIA,FELIPE MAIA,NaN,M,949b23d23985a4ef93de4f43d0eff8fa,991898e8-c29b-48f3-b063-993647de89f4
2,None,2015-02-01,None,None,None,AC,Titular,8b6b072f-2ddb-42cb-bde3-d1aae3fb5610,b27beba7-ca02-4041-a9e0-1793bcd141fe,59cd5d76-8acd-420c-af62-85fcdba59337,NaN,1976-10-23,dep.alanrick@camara.gov.br,178836,ALAN RICK MIRANDA,ALAN RICK,"Administrador de Empresas, Jornalista",M,3f8ce45065ead34564ffbb987a294be7,8b6b072f-2ddb-42cb-bde3-d1aae3fb5610
3,None,2015-02-01,None,None,None,MA,Titular,8e0752c7-21d0-46b9-8fef-0a3a49305652,b27beba7-ca02-4041-a9e0-1793bcd141fe,8a1caba9-c718-4aab-a664-425ca788d487,NaN,1973-11-11,dep.joaomarcelosouza@camara.gov.br,112437,JOÃO MARCELO SANTOS SOUZA,JOÃO MARCELO SOUZA,Psicólogo,M,de549e980bd660b3b287d5893a2e41a8,8e0752c7-21d0-46b9-8fef-0a3a49305652
4,None,2015-02-01,None,None,None,SP,Titular,b7e9e1fb-c94d-4e3e-ace5-eae5336202d8,b27beba7-ca02-4041-a9e0-1793bcd141fe,7cb50203-c46c-48e3-9d0a-76fdb27baa48,NaN,1950-05-20,dep.vanderleimacris@camara.gov.br,141553,VANDERLEI MACRIS,VANDERLEI MACRIS,NaN,M,0686f3aacaafeec85d46c3f48ceca6aa,b7e9e1fb-c94d-4e3e-ace5-eae5336202d8


In [11]:
# df2.to_csv('datasets/membership_with_role_deputado_and_foaf-55.csv', sep=';', encoding='utf-8')

In [23]:

index = df2['cam:dataFim'].isnull().values
current_from_memberships = set(df2.loc[index, 'cam:nomeCivil'].values)
current_from_xml = set(df_current_congressman['nome'].values)

print(len(current_from_memberships), len(current_from_xml))
current_from_memberships - current_from_xml

513 513


{'ELIZEU DIONIZIO SOUZA  DA SILVA', 'JOAO HENRIQUE HOLANDA  CALDAS'}

In [24]:
current_from_xml - current_from_memberships

{'ELIZEU DIONIZIO SOUZA DA SILVA', 'JOAO HENRIQUE HOLANDA CALDAS'}